In [75]:
import numpy as np
import numpy.matlib as mt
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.trials import align_trials
from ephysvibe.task import task_constants
import os 
from matplotlib import cm
from matplotlib import pyplot as plt
import glob
import pickle

from itertools import permutations
seed = 2023

In [12]:
def moving_average(data:np.ndarray,win:int, step:int=1)-> np.ndarray:
    d_shape=data.shape
    d_avg = np.zeros((d_shape[0],int(np.floor(d_shape[1]/step))))
    count = 0
    for i_step in np.arange(0,d_shape[1]-step,step):
        d_avg[:,count] = np.mean(data[:,i_step:i_step+win],axis=1)
        count +=1
    return d_avg

In [4]:
directory_b1    =   "W:/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure/session_struct/"
bhv_directory   =   os.path.normpath(str(directory_b1) +  "/bhv/")

area="pfc"
neurons_pfc_directory =   os.path.normpath(str(directory_b1) + area + "/neurons/*.h5")
neurons_pfc_files     =   glob.glob(neurons_pfc_directory, recursive=True)

area="v4"
neurons_v4_directory =   os.path.normpath(str(directory_b1) + area + "/neurons/*.h5")
neurons_v4_files     =   glob.glob(neurons_v4_directory, recursive=True)

area="lip"

neurons_lip_directory =   os.path.normpath(str(directory_b1) + area + "/neurons/*.h5")
neurons_lip_files     =   glob.glob(neurons_lip_directory, recursive=True)

In [9]:
win=100
code=1
select_block=1
time_before_sample  =   400
time_before_t1      =   450

timetotal_sample=   time_before_sample+450*3
timetotal_t1    =   time_before_t1+450*2


pfc_paths=neurons_pfc_files
numcells=1#len(neurons_pfc_files)


for p in pfc_paths[:numcells]:
    # print(str(n) + "/" + str(len(pfc_paths)))
    pfc_good, pfc_mua = 1, 1
    
    neu_data    =   NeuronData.from_python_hdf5(p)
    date_time   =   neu_data.date_time


    sp_sample_in_on,mask_sample_in = align_trials.align_on(
            sp_samples=neu_data.sp_samples,
            code_samples=neu_data.code_samples,
            code_numbers=neu_data.code_numbers,
            trial_error=neu_data.trial_error,
            block=neu_data.block,
            pos_code=neu_data.pos_code,
            select_block= select_block,
            select_pos= code,
            event ="sample_on",
            time_before = time_before_sample,
            error_type= 0,
        )
    
    pfc_sample_in_avg_sp   =   moving_average(data=sp_sample_in_on[:, :timetotal_sample],win=win, step=step)[:,:-win]

array([ 56, 156,  35, 142,  28,   2,  94, 140,  97,  44,  83,  16,  73,
        98, 131, 155, 164,  72, 106,  12, 158,  76,  14,  87, 163, 135,
        43,  48,  68,  53,  29, 147,  18,  99, 146, 114, 120,   1,  82,
        36, 149, 100,  81, 101,   5, 171, 125, 138,  17,  47,  86, 127,
        85,  52,  23, 166, 122,  62, 111, 117, 161,  66, 152,  15,  10,
        95,  84,  57,  79,  22,  33, 104, 153,  91,   6, 157, 134],
      dtype=int64)

In [28]:
step=1
win=100
pfc_sample_in_avg_sp   =   moving_average(data=sp_sample_in_on[:, :timetotal_sample],win=win, step=step)[:,:-win]

o1trials    =   np.where(np.floor(neu_data.sample_id[mask_sample_in]/10)==1)
o5trials    =   np.where(np.floor(neu_data.sample_id[mask_sample_in]/10)==5)

c1trials    =   np.where(neu_data.sample_id[mask_sample_in]%10==1)
c5trials    =   np.where(neu_data.sample_id[mask_sample_in]%10==5)

ntrials     =   np.where(neu_data.sample_id[mask_sample_in]==0)
nntrials    =   np.where(neu_data.sample_id[mask_sample_in]!=0)


min_c_trials=   np.min([c1trials[0].shape, c5trials[0].shape])
min_o_trials=   np.min([o1trials[0].shape, o5trials[0].shape])

o1_train     =   80
o1_test      =   20

tmp    =   np.random.permutation(o1trials[0])

o1_train_trials=tmp[:np.floor(min_c_trials*80/100)]

In [109]:
# tmp    =   np.random.permutation(o1trials[0])
int(np.floor(min_c_trials*80/100))
train=tmp[:int((min_c_trials*80/100))]
test=tmp[int((min_c_trials*80/100)):]

test_rep=mt.repmat(test, train.shape[0],1)
train_rep=mt.repmat(train, test.shape[0],1)
if np.where(test_rep-np.transpose(train_rep)==0)[0].shape[0]!=0:
    print("Warnings: similar trials for training and testing")

traintrials =   np.random.choice(train, train.shape)
testtrials  =   np.random.choice(test, test.shape)

array([  1,   2,  12,  12,  12,  14,  15,  15,  16,  16,  16,  33,  36,
        36,  43,  44,  47,  52,  56,  57,  73,  73,  82,  84,  85,  85,
        87,  94,  94,  98, 100, 100, 101, 101, 101, 106, 106, 111, 134,
       138, 138, 138, 140, 153, 156, 163, 164, 164, 166, 166, 166],
      dtype=int64)

In [78]:
test_rep=mt.repmat(test, train.shape[0],1)
train_rep=mt.repmat(train, test.shape[0],1)

In [26]:
mintrials=np.min([c1trials[0].shape, c5trials[0].shape])
mintrials


64